# Preambule
#### Goal
The goal of this session is to get familiar with the Bloomberg Python API.<br> 
This will be done by building a class containing a function which mimicks the behavior of the BDH Excel function.

#### What the function will do
Our BDH-like function should be able to : <br>
1 - Retrieve historical data <br>
2 - For as many tickers as possible <br>
3 - For as many fields as possible <br>
4 - Include the various options <br>
5 - And allow for the possibility to add overrides <br>

#### References 
https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

# I. Dependencies
These are the libraries we will be using in this notebook. blpapi is the library used for Bloomberg data.

In [20]:
import blpapi
import pandas as pd
import numpy as np
import datetime as dt
blpapi.__version__

'3.18.3'

# II. Set up the Bloomberg names

We here create variables using the Name class within blpapi. <br> 
This will allow to write cleaner and more concise code when refering to strings with the api.<br>
Below are only the names required for our present work. Many more exist and you can refer to the different examples within the SDK for ones of interest to your task.

In [2]:
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

# III. The BLP class
We now start to build our function within a dedicated class.<br>

A brief reminder on the class object in Python:<br>
- Classes must have a function called _\_init_\_() which is automatically executed at class initiation
- Classes can have one or several methods
- Class object need to be instaciated before using its methods

#### A. The init function

This function aims at starting the session and setting up the desired service 

#### B. The close session method:
Simply kills the session so no ghost connection remains. 

#### C. The BDP method:
3 steps: <br>
1- Create request<br>
2- Send request <br>
3- Extract data<br>




## Squelette code

1) boucle sur les msg (ticker) lst_msg

    2) boucle sur les dates fields data
    
        3) boucle sur les fields field
        
Transformer un dict en dataframe pd.DataFrame.Fromdict(nom dict)
        
Dict1[field] [ticker]

   -> Dict2[date]
   
   

        
        # recupere partie en bleu cf slides
        print(msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue())
        # renvoie la securities
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA))
        #info pour faire boucle sur les dates (car renvoie un nombre: le nombre de date)
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).numValues())
        # info pour boucle sur data, on recuperer le ieme bloc preciser en fonction (idee: faire une boucle et mettre i ou j a la place du 1)
        #recup date et faire une boucle de 1 a nb d'element 
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).getValue(15))
        #avoir la date
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).getValue(15).getElement(DATE).getValue())
        # nombre de fields
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).getValue(15).numElements())
        # avoir le name du fill
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).getValue(15).getElement(2).name())
        # avoir la value du fill
        print(msg.getElement(SECURITY_DATA).getElement(FIELD_DATA).getValue(15).getElement(2).getValue())

In [108]:
class BLP():
    #-----------------------------------------------------------------------------------------------------    
    
    def __init__(self):
        """
            Improve this
            BLP object initialization
            Synchronus event handling
            
        """
        # Create Session object
        self.session = blpapi.Session()
        
        
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
        
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
        
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')

        print('Session open')
    
    #-----------------------------------------------------------------------------------------------------
    
    def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = 'EUR'):
        """
            Summary:
                HistoricalDataRequest ; 
        
                Gets historical data for a set of securities and fields

            Inputs:
                strSecurity: list of str : list of tickers
                strFields: list of str : list of fields, must be static fields (e.g. px_last instead of last_price)
                startdate: date
                enddate
                per: periodicitySelection; daily, monthly, quarterly, semiannually or annually
                perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
                curr: string, else default currency is used 
                Days: nonTradingDayFillOption : NON_TRADING_WEEKDAYS*, ALL_CALENDAR_DAYS or ACTIVE_DAYS_ONLY
                fill: nonTradingDayFillMethod :  PREVIOUS_VALUE, NIL_VALUE
                
                Options can be selected these are outlined in “Reference Services and Schemas Guide.”    
            
            Output:
                A list containing as many dataframes as requested fields
            # Partial response : 6
            # Response : 5
            
        """
           
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request slide 22
        request = self.refDataSvc.createRequest('HistoricalDataRequest')
        
        # Put field and securities in list is single value is passed
        if type(strFields) == str:
            strFields = [strFields]
            
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
    
        # Append list of securities
        for strF in strFields:
            request.append('fields', strF)
    
        for strS in strSecurity:
            request.append('securities', strS)
    
        # Set other parameters
        request.set('startDate', startdate.strftime('%Y%m%d'))
        request.set('endDate', enddate.strftime('%Y%m%d'))
        request.set('periodicitySelection', per)
        request.set('currency', curr)
        request.set('nonTradingDayFillMethod', fill)
        request.set('nonTradingDayFillOption', days)
        request.set('periodicityAdjustment', perAdj)
    
        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------

        requestID = self.session.sendRequest(request)
        print("Sending request")
        
        #-----------------------------------------------------------------------
        # Receive request
        #-----------------------------------------------------------------------
        
        
    
        list_msg=[]
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            #print(msg)
            list_msg.append(msg)
            
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break        
        
        #-----------------------------------------------------------------------
        # Exploit data 
        #-----------------------------------------------------------------------
        
        #creer le disct le plus global
        #create as many empty  dictionaries as field
        for field in strFields:
            globals()['dict_'+field]={}
        
        
        for msg in list_msg:

            # recupere partie en bleu cf slides le ticker
            ticker_name=msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue()
            # renvoie la securities
            field_data = msg.getElement(SECURITY_DATA).getElement(FIELD_DATA)
            
            
            for field in strFields:
                
                globals()['dict_'+field][ticker_name]={}

            for i in range(0, field_data.numValues()):

                #recup date et faire une boucle de 1 a nb d'element 
                fields = field_data.getValue(i)
                #avoir la date
                dt_date = fields.getElement(DATE).getValue()


                for j in range(1,fields.numElements()):
                    # avoir le name du fill
                    field_name =str(fields.getElement(j).name())
                    # avoir la value du fill
                    field_value = fields.getElement(j).getValue()
                    
                    globals()['dict_'+field_name][ticker_name][dt_date]=field_value
                
        dict_Security_Fields={}
        for field in strFields:
                
                dict_Security_Fields[field]=pd.DataFrame(globals()['dict_'+field])
        
        return dict_Security_Fields if len(strFields)>1 else dict_Security_Fields[field]
    
    #-----------------------------------------------------------------------------------------------------

    def closeSession(self):
        print("Session closed")
        self.session.stop()

# IV. Tests

In [112]:
blp = BLP()
strFields = ["PX_LAST", "NAME"]
tickers = ["GLE FP Equity", "TTE FP Equity"]
startDate = dt.datetime(2020,10,1) 
endDate = dt.datetime(2020,11,3)
prices = blp.bdh(strSecurity=tickers, strFields = strFields, startdate = startDate, enddate = endDate)
blp.closeSession()

Session open
Sending request


In [114]:
prices['NAME']

,GLE FP Equity,TTE FP Equity
